# **HOW TO TRAIN NEW MODEL (En to Vi)**



Phiên bản V1 nhằm chạy thử model TED anh - việt với các đặc điểm:

1 - Chạy  7 epoch

2 - Thực hiện trên máy local

3 - Chạy với tập train theo file mẫu en2de 


## Data Pre-processing

In [1]:
!pwd

/home/chinh/clc_fairseq


In [4]:
# # Download and prepare the data
!bash examples/translation/prepare-wmt14en2vi.sh


Cloning Moses github repository (for tokenization scripts)...
fatal: destination path 'mosesdecoder' already exists and is not an empty directory.
Cloning Subword NMT repository (for BPE pre-processing)...
fatal: destination path 'subword-nmt' already exists and is not an empty directory.
--2020-12-12 15:09:34--  https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz [following]
--2020-12-12 15:09:37--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connect

In [9]:
!TEXT="clc-tokenized.en-vi" \
&& fairseq-preprocess --source-lang en --target-lang vi \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir "clc-data-bin/tokenized.en-vi" \
    --workers 20

2020-12-12 15:16:56 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='clc-data-bin/tokenized.en-vi', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='en', srcdict=None, target_lang='vi', task='translation', tensorboard_logdir=None, testpref='clc-tokenized.en-vi/test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None, tpu=False, trainpref='clc-tokenized.en-

# Training

In [6]:
!mkdir -p "clc-checkpoints/fconv_wmt_en_vi"
!CUDA_VISIBLE_DEVICES=0 fairseq-train "clc-data-bin/tokenized.en-vi" \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 400 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir "clc-checkpoints/fconv_wmt_en_vi" \
    --max-epoch 7


2020-12-12 13:12:42 | INFO | fairseq_cli.train | {'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'tensorboard_logdir': None, 'wandb_project': None, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False}, 'common_eval': {'_name': None, 'path': None, 'post_process': None, 'quiet': False, 'model_overrides': '{}', 'results_path': None}, 'distributed_training': {'_name': None, 'distributed_world_size': 1, 'distributed_rank': 0, 'distributed_backend': 'nccl', 'distributed_init_method': None, 'distributed_port': -1, 'device_id': 0, 'local_rank': 0, 'distribut

Traceback (most recent call last):
  File "/usr/local/bin/fairseq-train", line 33, in <module>
    sys.exit(load_entry_point('fairseq', 'console_scripts', 'fairseq-train')())
  File "/home/chinh/clc_fairseq/fairseq_cli/train.py", line 392, in cli_main
    distributed_utils.call_main(cfg, main)
  File "/home/chinh/clc_fairseq/fairseq/distributed_utils.py", line 334, in call_main
    main(cfg, **kwargs)
  File "/home/chinh/clc_fairseq/fairseq_cli/train.py", line 69, in main
    task.load_dataset(valid_sub_split, combine=False, epoch=1)
  File "/home/chinh/clc_fairseq/fairseq/tasks/translation.py", line 296, in load_dataset
    self.datasets[split] = load_langpair_dataset(
  File "/home/chinh/clc_fairseq/fairseq/tasks/translation.py", line 75, in load_langpair_dataset
    raise FileNotFoundError(
FileNotFoundError: Dataset not found: valid (ed-iwslt-data-bin/tokenized.en-vi)


In [ ]:
!pwd

In [ ]:
# Evaluate
!mkdir -p ed-iwslt-data-bin/fconv_wmt_en_vi
!fairseq-generate \
    ed-iwslt-data-bin/tokenized.en-vi/ \
    --path ed-iwslt-checkpoints/fconv_wmt_en_vi/checkpoint_best.pt \
    --beam 5 --remove-bpe | tee ed-iwslt-data-bin/fconv_wmt_en_vi/gen.out

Compute BLEU score


In [ ]:
!grep ^H ed-iwslt-data-bin/fconv_wmt_en_vi/gen.out | cut -f3- > ed-iwslt-data-bin/fconv_wmt_en_vi/gen.out.sys
!grep ^T ed-iwslt-data-bin/fconv_wmt_en_vi/gen.out | cut -f2- > ed-iwslt-data-bin/fconv_wmt_en_vi/gen.out.ref
!fairseq-score --sys ed-iwslt-data-bin/fconv_wmt_en_vi/gen.out.sys --ref ed-iwslt-data-bin/fconv_wmt_en_vi/gen.out.ref


BLEU4 = 0.18, quá tệ

In [ ]:
from fairseq.models.transformer import TransformerModel
model = TransformerModel.from_pretrained(
  'iwslt-checkpoints-v2/fconv_wmt_en_vi',
  checkpoint_file='checkpoint_best.pt',
  bpe='subword_nmt',
  bpe_codes = 'iwslt-tokenized.en-vi/code'
)
print(model.translate('I am going to school'))